# 11 Convolution Layer

In [1]:
import tensorflow as tf
import numpy as np
import time
import os
import re

from PIL import Image as PI

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

    LABEL
        0 -> NORMAL (3,319)
        1 -> PNEUMOTHORAX (2,084)
        2 -> PNEUMONIA (4,277)

# hyper parameter

In [2]:
training_epochs = 500
batch_size = 20
image_resize = 128
image_width = image_resize
image_height = image_resize
initial_rate = 1e-5
label_len = 3

home_dir = "D:/Backup_data/jongkeun"

image_file = "/dir_images/190819_newTraining/"
test_image_file = "/dir_images/190309_new_test/"

label_file = "/csv/190819_newTraining.csv"
test_label_file = "/csv/190309_new_test.csv"

# label, test_label

In [3]:
label_dir = home_dir + label_file
test_label_dir = home_dir + test_label_file

# label_dir = os.getcwd() + "/csv/181126.csv"
# test_label_dir = os.getcwd() + "/csv/181126_test.csv"

f_label = open(label_dir, 'r')
f_test_label = open(test_label_dir, 'r')

label_data = f_label.readlines()
test_label_data = f_test_label.readlines()

f_label.close()
f_test_label.close()

label_array = np.array(np.reshape(label_data, [-1, label_len]), dtype=np.int32)
test_label_array = np.array(np.reshape(test_label_data, [-1, label_len]), dtype=np.int32)

print(len(label_array))
print(len(test_label_array))

9182
1018


# image, test_image

In [4]:
image_dir = home_dir + image_file
test_image_dir = home_dir + test_image_file

image_list = os.listdir(image_dir)
test_image_list = os.listdir(test_image_dir)

image_list = sorted(image_list, key=lambda x: (int(re.sub('\D','',x)),x))
test_image_list = sorted(test_image_list, key=lambda y: (int(re.sub('\D','',y)),y))

image_array = []
test_image_array = []

with tf.device('/gpu:0') :
    for i in range(len(image_list)) :
        image_list[i] = image_dir + image_list[i]
        image_array.append(np.array(PI.open(image_list[i]).resize((image_resize, image_resize)).convert('L')))

    for j in range(len(test_image_list)) :
        test_image_list[j] = test_image_dir + test_image_list[j]
        test_image_array.append(np.array(PI.open(test_image_list[j]).resize((image_resize, image_resize)).convert('L')))

    image_array = np.reshape(image_array, [-1, image_width*image_height])
    test_image_array = np.reshape(test_image_array, [-1, image_width*image_height])

    image_array = np.array(np.reshape(image_array, [-1, image_width, image_height, 1]), dtype=np.float32)
    test_image_array = np.array(np.reshape(test_image_array, [-1, image_width, image_height, 1]), dtype=np.float32)

print(len(image_array))
print(len(test_image_array))

9182
1018


# feeding value

In [5]:
x = tf.placeholder(dtype=tf.float32, shape=[None, image_width, image_height, 1], name='input_image')
y_ = tf.placeholder(dtype=tf.float32, shape=[None, label_len], name='input_label')
learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

is_training = tf.placeholder(dtype=tf.bool)

# learning layers

In [15]:
def learning_layer(input_data, size, dropout, is_training) :
    layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME')
    layer = tf.layers.max_pooling2d(layer, [2, 2], [2, 2], padding='SAME')
    layer = tf.layers.dropout(layer, dropout, is_training)
    
    return layer

with tf.device('/gpu:0') :
    layer0_1 = tf.layers.conv2d(x, 8, [3,3], padding='SAME')
    layer1 = learning_layer(input_data=layer0_1, size=256, dropout=0.7, is_training=is_training)
    
    layer1_1 = tf.layers.conv2d(layer1, 128, [3,3], padding='SAME')
    layer2 = learning_layer(input_data=layer1_1, size=512, dropout=0.7, is_training=is_training)
    
    layer2_1 = tf.layers.conv2d(layer2, 256, [3,3], padding='SAME')
    layer2_2 = tf.layers.conv2d(layer2, 256, [3,3], padding='SAME')
    layer3 = learning_layer(input_data=layer2_1, size=64, dropout=0.7, is_training=is_training)
    
    layer3_1 = tf.layers.conv2d(layer3, 128, [3,3], padding='SAME')
    layer3_2 = tf.layers.conv2d(layer3_1, 128, [3,3], padding='SAME')
    layer3_3 = tf.layers.conv2d(layer3_2, 123, [3,3], padding='SAME')
    layer4 = learning_layer(input_data=layer3_3, size=64, dropout=0.7, is_training=is_training)

In [16]:
fully_layer = tf.contrib.layers.flatten(layer4)
fully_layer = tf.layers.dense(fully_layer, 1024, activation=tf.nn.relu)
fully_layer = tf.layers.dropout(fully_layer, 0.5, is_training)

fully_layer2 = tf.layers.dense(fully_layer, 512, activation=tf.nn.relu)
fully_layer2 = tf.layers.dropout(fully_layer2, 0.5, is_training)

In [17]:
logits = tf.layers.dense(fully_layer2, label_len, activation=None)

with tf.name_scope('pred') : 
    pred = tf.nn.softmax(logits)
with tf.name_scope('loss') :
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))
    
with tf.device('/gpu:0') :
    with tf.name_scope('train') :
            train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

label_output = tf.argmax(pred, 1)
label_input = tf.argmax(y_, 1)

In [18]:
print(label_output)
print(label_input)

Tensor("ArgMax_6:0", shape=(?,), dtype=int64)
Tensor("ArgMax_7:0", shape=(?,), dtype=int64)


In [19]:
saver = tf.train.Saver()

tf.summary.scalar('scalar_loss', loss)

image_shaped_input = tf.reshape(x, [-1, image_resize, image_resize, 1])
tf.summary.image('input', image_shaped_input)

<tf.Tensor 'input_1:0' shape=() dtype=string>

In [20]:
config = tf.ConfigProto(log_device_placement = True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess :
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
    
#     f = open(log_dir+log_file, 'w')
    
#     saver = tf.train.import_meta_graph('meta_data_name')
#     saver.restore(sess, tf.train.latest_checkpoint('saved'))
    
    sess.run(tf.global_variables_initializer())
    
#     for var in tf.trainable_variables() :
#         tf.summary.histogram(var.name, var)

#     merged_summary = tf.summary.merge_all()  
#     train_writer = tf.summary.FileWriter(merge_dir, sess.graph)
    
    total_batch = int(len(image_list) / batch_size)
    correct_list = []
    incorrect_list = []
    cost_list = []
    cost_sum_list = []
    pred_list = []
    
#     sensitivity_acc = 0
#     specificity_acc = 0
    total_acc = 0
    cost_sum = 0
    cost_flag = 0
    rate_ = 1
    
    start_time = time.time()
    
    print("")
    
    for epoch in range(training_epochs) :
        total_cost = 0
        
        if epoch%50 == 49 :
                rate_ *= 0.8
                
        if epoch%10 == 9 :
#             rate_ *= 0.8
            midpoint = int(time.time() - start_time)
        
            print("")
            print("=======================================================================================")
            print('{:03d}:{:02d}:{:02d}'.format(midpoint//3600, (midpoint%3600//60), midpoint%60))
            print("=======================================================================================")
            print("")
        
        batch_index = np.random.choice(len(image_array), total_batch, replace=False)
                
        for i in range(total_batch) :                
            _, _loss = sess.run([train, loss], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]], 
                                                          is_training: True, learning_rate: initial_rate*rate_})    

#             _, _loss, _merge = sess.run([train, loss, merged_summary], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]],
#                                                                                   is_training: True, learning_rate: initial_rate*rate_})
            
#             saver.save(sess, save_dir, global_step=total_batch, write_meta_graph=False)
            
            total_cost += _loss

        avg_cost = total_cost/total_batch
        print('Epoch : ', '%4d' % (epoch + 1), '    Avg. cost = ', '{:.4f}'.format(avg_cost))            
        
#         train_writer.add_summary(_merge, epoch)
               
    print("")
    print("=======================================================================================")
    print("================================     Training done     ================================")
    print("=======================================================================================")
    print("")
    
    for test in range(len(test_image_array)) :
        _acc, _label_input, _label_output, _pred = sess.run([acc, label_input, label_output, pred], 
                             feed_dict={x: test_image_array[[test]], y_: test_label_array[[test]], is_training: False})

        total_acc += _acc
        pred_list.append(list(_pred[0]))
        
        print("n: ", test, "    label_input: ", _label_input, "    label_output: ", _label_output)
        
        if _label_input == _label_output :
            correct_list.append(test)   
        else :
            incorrect_list.append(test)
        
    print("")
#     print("       SENSITIVITY ACC : ", '{:.5f}'.format(sensitivity_acc / 200))
#     print("       SPECIFICITY ACC : ", '{:.5f}'.format(specificity_acc / 100))
    print("          TOTAL ACC    : ", '{:.5f}'.format(total_acc / len(test_image_array)))
    print("")
    print(pred_list)
    
#     for cnt in range(len(correct_list)) :
#         data = correct_list[cnt]
#         f.write(str(data)+"\n")
#         print(data)
    
#     tf.train.write_graph(sess.graph, './', pb_, as_text=False)
#     tf.train.write_graph(sess.graph, './', txt_, as_text=True)
#     train_writer.add_graph(sess.graph)
    
    end_time = int(time.time() - start_time)
    
    print("")
    print("=======================================================================================")
    print('{:03d}:{:02d}:{:02d}'.format(end_time//3600, (end_time%3600//60), end_time%60))
    print("=======================================================================================")
    print("")
    
#     for cnt_incorrect in range(len(incorrect_list)) :
#         print(incorrect_list[cnt_incorrect])
    
#     f.close()
    
    coord.request_stop()
    coord.join(thread)

W1129 15:45:06.664725 22592 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.



Epoch :     1     Avg. cost =  2.6712
Epoch :     2     Avg. cost =  1.4419
Epoch :     3     Avg. cost =  1.1463
Epoch :     4     Avg. cost =  0.9271
Epoch :     5     Avg. cost =  0.8027
Epoch :     6     Avg. cost =  0.6758
Epoch :     7     Avg. cost =  0.6799
Epoch :     8     Avg. cost =  0.7590
Epoch :     9     Avg. cost =  0.6844

000:01:36

Epoch :    10     Avg. cost =  0.6699
Epoch :    11     Avg. cost =  0.6707
Epoch :    12     Avg. cost =  0.6588
Epoch :    13     Avg. cost =  0.5918
Epoch :    14     Avg. cost =  0.6855
Epoch :    15     Avg. cost =  0.6457
Epoch :    16     Avg. cost =  0.5550
Epoch :    17     Avg. cost =  0.6541
Epoch :    18     Avg. cost =  0.5688
Epoch :    19     Avg. cost =  0.4707

000:03:22

Epoch :    20     Avg. cost =  0.5557
Epoch :    21     Avg. cost =  0.5800
Epoch :    22     Avg. cost =  0.5430
Epoch :    23     Avg. cost =  0.5190
Epoch :    24     Avg. cost =  0.5426
Epoch :    25     Avg. cost =  0.5279
Epoch :    26     Avg. co

Epoch :   148     Avg. cost =  0.0629
Epoch :   149     Avg. cost =  0.0766

000:27:03

Epoch :   150     Avg. cost =  0.0751
Epoch :   151     Avg. cost =  0.0479
Epoch :   152     Avg. cost =  0.0366
Epoch :   153     Avg. cost =  0.0304
Epoch :   154     Avg. cost =  0.0622
Epoch :   155     Avg. cost =  0.0457
Epoch :   156     Avg. cost =  0.0366
Epoch :   157     Avg. cost =  0.0232
Epoch :   158     Avg. cost =  0.0327
Epoch :   159     Avg. cost =  0.0339

000:28:53

Epoch :   160     Avg. cost =  0.0437
Epoch :   161     Avg. cost =  0.0418
Epoch :   162     Avg. cost =  0.0303
Epoch :   163     Avg. cost =  0.0565
Epoch :   164     Avg. cost =  0.0632
Epoch :   165     Avg. cost =  0.0299
Epoch :   166     Avg. cost =  0.0120
Epoch :   167     Avg. cost =  0.0362
Epoch :   168     Avg. cost =  0.0361
Epoch :   169     Avg. cost =  0.0288

000:30:43

Epoch :   170     Avg. cost =  0.0557
Epoch :   171     Avg. cost =  0.0163
Epoch :   172     Avg. cost =  0.0226
Epoch :   173 

Epoch :   290     Avg. cost =  0.0014
Epoch :   291     Avg. cost =  0.0004
Epoch :   292     Avg. cost =  0.0007
Epoch :   293     Avg. cost =  0.0188
Epoch :   294     Avg. cost =  0.0020
Epoch :   295     Avg. cost =  0.0007
Epoch :   296     Avg. cost =  0.0004
Epoch :   297     Avg. cost =  0.0003
Epoch :   298     Avg. cost =  0.0001
Epoch :   299     Avg. cost =  0.0004

000:54:36

Epoch :   300     Avg. cost =  0.0002
Epoch :   301     Avg. cost =  0.0053
Epoch :   302     Avg. cost =  0.0002
Epoch :   303     Avg. cost =  0.0002
Epoch :   304     Avg. cost =  0.0003
Epoch :   305     Avg. cost =  0.0011
Epoch :   306     Avg. cost =  0.0004
Epoch :   307     Avg. cost =  0.0004
Epoch :   308     Avg. cost =  0.1015
Epoch :   309     Avg. cost =  0.0003

000:56:26

Epoch :   310     Avg. cost =  0.0010
Epoch :   311     Avg. cost =  0.0004
Epoch :   312     Avg. cost =  0.0016
Epoch :   313     Avg. cost =  0.0015
Epoch :   314     Avg. cost =  0.0168
Epoch :   315     Avg. cos

Epoch :   437     Avg. cost =  0.0000
Epoch :   438     Avg. cost =  0.0000
Epoch :   439     Avg. cost =  0.0000

001:20:15

Epoch :   440     Avg. cost =  0.0000
Epoch :   441     Avg. cost =  0.0000
Epoch :   442     Avg. cost =  0.0000
Epoch :   443     Avg. cost =  0.0000
Epoch :   444     Avg. cost =  0.0000
Epoch :   445     Avg. cost =  0.0000
Epoch :   446     Avg. cost =  0.0000
Epoch :   447     Avg. cost =  0.0000
Epoch :   448     Avg. cost =  0.0000
Epoch :   449     Avg. cost =  0.0000

001:22:05

Epoch :   450     Avg. cost =  0.0000
Epoch :   451     Avg. cost =  0.0000
Epoch :   452     Avg. cost =  0.0000
Epoch :   453     Avg. cost =  0.0000
Epoch :   454     Avg. cost =  0.0000
Epoch :   455     Avg. cost =  0.0000
Epoch :   456     Avg. cost =  0.0000
Epoch :   457     Avg. cost =  0.0000
Epoch :   458     Avg. cost =  0.0272
Epoch :   459     Avg. cost =  0.0000

001:23:55

Epoch :   460     Avg. cost =  0.0000
Epoch :   461     Avg. cost =  0.0000
Epoch :   462 

...

n:  250     label_input:  [0]     label_output:  [0]
n:  251     label_input:  [0]     label_output:  [0]
n:  252     label_input:  [0]     label_output:  [0]
n:  253     label_input:  [0]     label_output:  [1]
n:  254     label_input:  [0]     label_output:  [0]
n:  255     label_input:  [0]     label_output:  [0]
n:  256     label_input:  [0]     label_output:  [0]
n:  257     label_input:  [0]     label_output:  [1]
n:  258     label_input:  [0]     label_output:  [0]
n:  259     label_input:  [0]     label_output:  [0]
n:  260     label_input:  [0]     label_output:  [0]
n:  261     label_input:  [0]     label_output:  [0]
n:  262     label_input:  [0]     label_output:  [0]
n:  263     label_input:  [0]     label_output:  [0]
n:  264     label_input:  [0]     label_output:  [1]
n:  265     label_input:  [0]     label_output:  [0]
n:  266     label_input:  [0]     label_output:  [0]
n:  267     label_input:  [0]     label_output:  [0]
n:  268     label_input:  [0]     label_output

n:  416     label_input:  [1]     label_output:  [1]
n:  417     label_input:  [1]     label_output:  [1]
n:  418     label_input:  [1]     label_output:  [1]
n:  419     label_input:  [1]     label_output:  [1]
n:  420     label_input:  [1]     label_output:  [1]
n:  421     label_input:  [1]     label_output:  [1]
n:  422     label_input:  [1]     label_output:  [1]
n:  423     label_input:  [1]     label_output:  [1]
n:  424     label_input:  [1]     label_output:  [1]
n:  425     label_input:  [1]     label_output:  [1]
n:  426     label_input:  [1]     label_output:  [1]
n:  427     label_input:  [1]     label_output:  [1]
n:  428     label_input:  [1]     label_output:  [1]
n:  429     label_input:  [1]     label_output:  [1]
n:  430     label_input:  [1]     label_output:  [1]
n:  431     label_input:  [1]     label_output:  [1]
n:  432     label_input:  [1]     label_output:  [1]
n:  433     label_input:  [1]     label_output:  [1]
n:  434     label_input:  [1]     label_output

n:  582     label_input:  [2]     label_output:  [2]
n:  583     label_input:  [2]     label_output:  [2]
n:  584     label_input:  [2]     label_output:  [2]
n:  585     label_input:  [2]     label_output:  [2]
n:  586     label_input:  [2]     label_output:  [2]
n:  587     label_input:  [2]     label_output:  [2]
n:  588     label_input:  [2]     label_output:  [2]
n:  589     label_input:  [2]     label_output:  [2]
n:  590     label_input:  [2]     label_output:  [2]
n:  591     label_input:  [2]     label_output:  [2]
n:  592     label_input:  [2]     label_output:  [2]
n:  593     label_input:  [2]     label_output:  [2]
n:  594     label_input:  [2]     label_output:  [2]
n:  595     label_input:  [2]     label_output:  [2]
n:  596     label_input:  [2]     label_output:  [2]
n:  597     label_input:  [2]     label_output:  [2]
n:  598     label_input:  [2]     label_output:  [2]
n:  599     label_input:  [2]     label_output:  [2]
n:  600     label_input:  [2]     label_output

n:  745     label_input:  [2]     label_output:  [2]
n:  746     label_input:  [2]     label_output:  [2]
n:  747     label_input:  [2]     label_output:  [2]
n:  748     label_input:  [2]     label_output:  [2]
n:  749     label_input:  [2]     label_output:  [2]
n:  750     label_input:  [2]     label_output:  [2]
n:  751     label_input:  [2]     label_output:  [2]
n:  752     label_input:  [2]     label_output:  [2]
n:  753     label_input:  [2]     label_output:  [2]
n:  754     label_input:  [2]     label_output:  [2]
n:  755     label_input:  [2]     label_output:  [2]
n:  756     label_input:  [2]     label_output:  [2]
n:  757     label_input:  [2]     label_output:  [2]
n:  758     label_input:  [2]     label_output:  [2]
n:  759     label_input:  [2]     label_output:  [2]
n:  760     label_input:  [2]     label_output:  [2]
n:  761     label_input:  [2]     label_output:  [2]
n:  762     label_input:  [2]     label_output:  [2]
n:  763     label_input:  [2]     label_output

n:  912     label_input:  [2]     label_output:  [0]
n:  913     label_input:  [2]     label_output:  [2]
n:  914     label_input:  [2]     label_output:  [2]
n:  915     label_input:  [2]     label_output:  [2]
n:  916     label_input:  [2]     label_output:  [2]
n:  917     label_input:  [2]     label_output:  [2]
n:  918     label_input:  [2]     label_output:  [2]
n:  919     label_input:  [2]     label_output:  [2]
n:  920     label_input:  [2]     label_output:  [2]
n:  921     label_input:  [2]     label_output:  [0]
n:  922     label_input:  [2]     label_output:  [2]
n:  923     label_input:  [2]     label_output:  [0]
n:  924     label_input:  [2]     label_output:  [2]
n:  925     label_input:  [2]     label_output:  [2]
n:  926     label_input:  [2]     label_output:  [2]
n:  927     label_input:  [2]     label_output:  [2]
n:  928     label_input:  [2]     label_output:  [2]
n:  929     label_input:  [2]     label_output:  [0]
n:  930     label_input:  [2]     label_output